## Midterm Project

**Prepared By: Group 7**
- Hozefa Patel [N01686385]
- Jenil Pancholi [N01665133]
- Dev Ariwala [N01664568]

### Step 1: Import Libraries and Set Seed

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.datasets import KMNIST
from torch.utils.data import DataLoader
import numpy as np
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import ParameterGrid

torch.manual_seed(7)  # Set seed for reproducibility
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
EPOCHS = 50  # Adjust epochs for demonstration
BATCH_SIZE = 64  # Batch size

### Step 2: Load and Preprocess the KMNIST Dataset

In [ ]:
# Load training dataset into a single batch to compute mean and stddev.
transform = transforms.Compose([transforms.ToTensor()])
trainset = KMNIST(root='./pt_data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=len(trainset), shuffle=True)
data = next(iter(trainloader))
mean = data[0].mean()
stddev = data[0].std()

# Helper function needed to standardize data when loading datasets.
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, stddev)
])

trainset = KMNIST(root='./pt_data', train=True, download=True, transform=transform)
testset = KMNIST(root='./pt_data', train=False, download=True, transform=transform)

trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
testloader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False)

100%|██████████| 18165135/18165135 [00:13<00:00, 1323150.90it/s]


Extracting ./pt_data/KMNIST/raw/train-images-idx3-ubyte.gz to ./pt_data/KMNIST/raw



100%|██████████| 29497/29497 [00:00<00:00, 206311.07it/s]


Extracting ./pt_data/KMNIST/raw/train-labels-idx1-ubyte.gz to ./pt_data/KMNIST/raw



100%|██████████| 3041136/3041136 [00:03<00:00, 841747.25it/s]


Extracting ./pt_data/KMNIST/raw/t10k-images-idx3-ubyte.gz to ./pt_data/KMNIST/raw



100%|██████████| 5120/5120 [00:00<00:00, 15123124.28it/s]


Extracting ./pt_data/KMNIST/raw/t10k-labels-idx1-ubyte.gz to ./pt_data/KMNIST/raw



### Step 3: Define the Model

In [ ]:
class ANNModel(nn.Module):
    def __init__(self):
        super(ANNModel, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28 * 28, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(256, 128)
        self.bn3 = nn.BatchNorm1d(128)
        self.dropout3 = nn.Dropout(0.5)
        self.fc4 = nn.Linear(128, 64)
        self.bn4 = nn.BatchNorm1d(64)
        self.fc5 = nn.Linear(64, 10)

    def forward(self, x):
        x = self.flatten(x)
        x = self.dropout1(nn.ReLU()(self.bn1(self.fc1(x))))
        x = self.dropout2(nn.ReLU()(self.bn2(self.fc2(x))))
        x = self.dropout3(nn.ReLU()(self.bn3(self.fc3(x))))
        x = nn.ReLU()(self.bn4(self.fc4(x)))
        x = self.fc5(x)
        return x

model = ANNModel().to(device)

### Step 4: Define Training and Evaluation Functions

In [ ]:
def train_model(model, trainloader, optimizer, loss_function, epochs):
    model.to(device)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1)
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_batches = 0

        for inputs, targets in trainloader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_function(outputs, targets)
            loss.backward()
            optimizer.step()

            _, indices = torch.max(outputs.data, 1)
            train_correct += (indices == targets).sum().item()
            train_batches += 1
            train_loss += loss.item()

        train_loss /= train_batches
        train_acc = 100. * train_correct / (train_batches * BATCH_SIZE)
        scheduler.step()
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.2f}%")

    return model

def evaluate_model(model, testloader, loss_function):
    model.eval()
    test_loss = 0.0
    test_correct = 0
    test_batches = 0

    with torch.no_grad():
        for inputs, targets in testloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = loss_function(outputs, targets)
            _, indices = torch.max(outputs, 1)
            test_correct += (indices == targets).sum().item()
            test_batches += 1
            test_loss += loss.item()

    test_loss /= test_batches
    test_acc = 100. * test_correct / (test_batches * BATCH_SIZE)
    return test_loss, test_acc

### Step 5: Perform Hyperparameter Tuning and Evaluation

In [ ]:
# Define the hyperparameters grid
param_grid = {
    'optimizer': ['adam', 'rmsprop', 'adamw'],
    'learning_rate': [0.001]
}

param_combinations = list(ParameterGrid(param_grid))

best_acc = 0.0
best_params = None
results = []

for params in param_combinations:
    print(f"Testing combination: {params}")
    model = ANNModel().to(device)  # Reset the model for each combination
    if params['optimizer'] == 'adam':
        optimizer = torch.optim.Adam(model.parameters(), lr=params['learning_rate'])
    elif params['optimizer'] == 'rmsprop':
        optimizer = torch.optim.RMSprop(model.parameters(), lr=params['learning_rate'])
    elif params['optimizer'] == 'adamw':
        optimizer = torch.optim.AdamW(model.parameters(), lr=params['learning_rate'])

    # Train the model
    trained_model = train_model(model, trainloader, optimizer, nn.CrossEntropyLoss(), EPOCHS)

    # Evaluate the model
    test_loss, test_acc = evaluate_model(trained_model, testloader, nn.CrossEntropyLoss())
    print(f"Test Accuracy: {test_acc:.2f}%")

    results.append({
        'optimizer': params['optimizer'],
        'learning_rate': params['learning_rate'],
        'test_accuracy': test_acc
    })

    if test_acc > best_acc:
        best_acc = test_acc
        best_params = params

# Convert results to DataFrame and display
results_df = pd.DataFrame(results)
print(results_df)

print(f"Best Test Accuracy: {best_acc:.2f}% with parameters: {best_params}")

Testing combination: {'learning_rate': 0.001, 'optimizer': 'adam'}
Epoch 1/50, Train Loss: 0.7256, Train Accuracy: 78.48%
Epoch 2/50, Train Loss: 0.4232, Train Accuracy: 87.21%
Epoch 3/50, Train Loss: 0.3558, Train Accuracy: 89.27%
Epoch 4/50, Train Loss: 0.3142, Train Accuracy: 90.46%
Epoch 5/50, Train Loss: 0.2878, Train Accuracy: 91.25%
Epoch 6/50, Train Loss: 0.2692, Train Accuracy: 91.89%
Epoch 7/50, Train Loss: 0.2493, Train Accuracy: 92.54%
Epoch 8/50, Train Loss: 0.2345, Train Accuracy: 92.96%
Epoch 9/50, Train Loss: 0.2189, Train Accuracy: 93.38%
Epoch 10/50, Train Loss: 0.2097, Train Accuracy: 93.50%
Epoch 11/50, Train Loss: 0.1995, Train Accuracy: 93.96%
Epoch 12/50, Train Loss: 0.1946, Train Accuracy: 94.10%
Epoch 13/50, Train Loss: 0.1817, Train Accuracy: 94.43%
Epoch 14/50, Train Loss: 0.1749, Train Accuracy: 94.69%
Epoch 15/50, Train Loss: 0.1703, Train Accuracy: 94.86%
Epoch 16/50, Train Loss: 0.1487, Train Accuracy: 95.45%
Epoch 17/50, Train Loss: 0.1381, Train Accurac

### Step 6: Display Results

In [ ]:
from IPython.display import display

# Display the results in a table format
results_df = pd.DataFrame(results)
display(results_df)

print(f"Best Test Accuracy: {best_acc:.2f}% with parameters: {best_params}")

,optimizer,learning_rate,test_accuracy
0,adam,0.001,91.859076
1,rmsprop,0.001,91.679936
2,adamw,0.001,92.018312


Best Test Accuracy: 92.02% with parameters: {'learning_rate': 0.001, 'optimizer': 'adamw'}
